In [25]:
import pandas as pd
df11 = pd.read_csv("../data/dailyActivity_merged34.csv")
df22 = pd.read_csv("../data/dailyActivity_merged45.csv")

df33 = pd.concat([df11, df22])

df33.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [27]:
df33

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.500000,8.500000,0.0,1.88,0.55,6.06,0.00,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.970000,6.970000,0.0,1.57,0.69,4.71,0.00,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.740000,6.740000,0.0,2.44,0.40,3.91,0.00,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.280000,6.280000,0.0,2.14,1.26,2.83,0.00,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.160000,8.160000,0.0,2.71,0.41,5.04,0.00,36,10,221,773,1863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,8877689391,5/8/2016,10686,8.110000,8.110000,0.0,1.08,0.20,6.80,0.00,17,4,245,1174,2847
936,8877689391,5/9/2016,20226,18.250000,18.250000,0.0,11.10,0.80,6.24,0.05,73,19,217,1131,3710
937,8877689391,5/10/2016,10733,8.150000,8.150000,0.0,1.35,0.46,6.28,0.00,18,11,224,1187,2832
938,8877689391,5/11/2016,21420,19.559999,19.559999,0.0,13.22,0.41,5.89,0.00,88,12,213,1127,3832


In [28]:
df33.shape

(1880, 15)

In [30]:
df33["ActivityDate"] = pd.to_datetime(df33["ActivityDate"], errors="coerce")

df33[["ActivityDate"]]

,ActivityDate
0,2016-04-12
1,2016-04-13
2,2016-04-14
3,2016-04-15
4,2016-04-16
...,...
935,2016-05-08
936,2016-05-09
937,2016-05-10
938,2016-05-11


In [31]:
df33["ActivityDate"].dtype

dtype('<M8[ns]')

In [ ]:
df33.duplicated(subset=["Id", "ActivityDate"]).sum() # 겹치는게 있구나!!

np.int64(940)

In [33]:
df33["TotalActiveMinutes"] = (
    df33["VeryActiveMinutes"]
    + df33["FairlyActiveMinutes"]
    + df33["LightlyActiveMinutes"]
)

df33[[
    "VeryActiveMinutes",
    "FairlyActiveMinutes",
    "LightlyActiveMinutes",
    "TotalActiveMinutes"
]].head()

,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,TotalActiveMinutes
0,25,13,328,366
1,21,19,217,257
2,30,11,181,222
3,29,34,209,272
4,36,10,221,267


In [34]:
df33["SedentaryRatio"] = df33["SedentaryMinutes"] / 1440

df33[["SedentaryMinutes", "SedentaryRatio"]].head()

,SedentaryMinutes,SedentaryRatio
0,728,0.505556
1,776,0.538889
2,1218,0.845833
3,726,0.504167
4,773,0.536806


In [35]:
df33["IntensityScore"] = (
    3 * df33["VeryActiveMinutes"]
    + 2 * df33["FairlyActiveMinutes"]
    + 1 * df33["LightlyActiveMinutes"]
)

df33[[
    "VeryActiveMinutes",
    "FairlyActiveMinutes",
    "LightlyActiveMinutes",
    "IntensityScore"
]].head()

,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,IntensityScore
0,25,13,328,429
1,21,19,217,318
2,30,11,181,293
3,29,34,209,364
4,36,10,221,349


In [36]:
df33["CaloriesPerStep"] = df33["Calories"] / (df33["TotalSteps"] + 1)

df33[["TotalSteps", "Calories", "CaloriesPerStep"]].head()

,TotalSteps,Calories,CaloriesPerStep
0,13162,1985,0.150801
1,10735,1797,0.167381
2,10460,1776,0.169773
3,9762,1745,0.178736
4,12669,1863,0.147040
